In [1]:
import os
import gc
from datetime import date

import torch
from tqdm.notebook import tqdm
import pandas as pd
from langchain.schema import Document

from utils import (
    compute_metrics,
    load_all_available_transcripts,
    SummarizationPipeline,
    TextChunker,
    LoggingConfig,
    ModelConfig,
    TopicModeler,
    Retriever
)

/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
transcripts = load_all_available_transcripts()
transcripts = transcripts[:1]

2025-05-04 19:36:38,711 - utils.loaders - INFO - 32307
2025-05-04 19:36:38,847 - utils.loaders - INFO - Successfully loaded local transcripts for 32307
2025-05-04 19:36:38,849 - utils.loaders - INFO - Successfully loaded 32307.csv
2025-05-04 19:36:38,849 - utils.loaders - INFO - 126475
2025-05-04 19:36:38,882 - utils.loaders - INFO - Successfully loaded local transcripts for 126475
2025-05-04 19:36:38,883 - utils.loaders - INFO - Successfully loaded 126475.csv
2025-05-04 19:36:38,884 - utils.loaders - INFO - 26446
2025-05-04 19:36:38,900 - utils.loaders - INFO - Successfully loaded local transcripts for 26446
2025-05-04 19:36:38,901 - utils.loaders - INFO - Successfully loaded 26446.csv
2025-05-04 19:36:38,902 - utils.loaders - INFO - 388904
2025-05-04 19:36:38,928 - utils.loaders - INFO - Successfully loaded local transcripts for 388904
2025-05-04 19:36:38,929 - utils.loaders - INFO - Successfully loaded 388904.csv
2025-05-04 19:36:38,929 - utils.loaders - INFO - 312932093
2025-05-04 

In [3]:
transcripts.head()

,companyid,companyname,mostimportantdateutc,mostimportanttimeutc,headline,full_text,uuid,word_count,word_count_nltk
0,32307,NVIDIA Corporation,2023-01-05,15:50:00,NVIDIA Corporation Presents at J.P. MORGAN 21S...,Analysts: All right. Why don't we go ahead and...,ca419775-262e-4e3a-975a-e4ead13ef55b,9541,10911


In [4]:
original_texts = transcripts['full_text'].tolist()
metadata = transcripts[['uuid', 'companyid', 'companyname', 'word_count_nltk']]

In [19]:
checkpoints = [
    'facebook/bart-large-cnn',
    'google-t5/t5-base',
    'google/pegasus-x-large',
    'human-centered-summarization/financial-summarization-pegasus',
]

In [20]:
logging_config = LoggingConfig()
all_metrics = []

In [21]:
df = pd.read_csv(
    'hf://datasets/sohomghosh/FinRAD_Financial_Readability_Assessment_Dataset/FinRAD_13K_terms_definitions_labels.csv'
)
df = df[['terms', 'definitions', 'source', 'assigned_readability']]
df = df.dropna(subset=['definitions'])
df['combined'] = df['terms'] + ': ' + df['definitions']

retriever = Retriever(df.combined.tolist(), 5)

Indexing: 100%|██████████| 13112/13112 [00:00<00:00, 507541.87vec/s]


In [22]:
for checkpoint in checkpoints:
    model_config = ModelConfig(
        model_name_or_path=checkpoint,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    pipeline = SummarizationPipeline(
        model_config=model_config,
        logging_config=logging_config,
        remote=True
    )
    tokenizer = pipeline.get_tokenizer()
    chunker = TextChunker(tokenizer)

    summaries = []
    for text in tqdm(original_texts, desc=f"Summarizing with {checkpoint}"):
        chunks = chunker.chunk_text(text)

        print([len(x) for x in chunks])

        chunker.resize_chunks(int(chunker._adjusted_chunk_size / 4))

        print(f'chunk size: {chunker._adjusted_chunk_size}')

        tm_chunks = chunker.chunk_text(text)

        print([len(x) for x in tm_chunks])

        chunker.resize_chunks(int(chunker._adjusted_chunk_size * 4))

        tm = TopicModeler(
            chunks=[Document(page_content=doc) for doc in tm_chunks], 
            speed='learn', 
            workers=8
        )
        topic_words, _, topic_nums = tm.get_topics(1)

        for words, tid in zip(topic_words, topic_nums):
            print(f'Topic: ' + ' '.join(words))

        topics_string = ' '.join(words)
        top_results, _ = retriever.search(topics_string, 3)

        for result in top_results:
            print(f'found definition: {result}')

        chunks.insert(0, 'context: ' + ', '.join(top_results))
        chunk_summaries = [pipeline.summarize(c) for c in chunks]
        combined = " ".join(chunk_summaries)

        # If combined summary is too long, iteratively re-chunk and re-summarize
        max_rounds = 5
        for _ in range(max_rounds):
            tokens = tokenizer(combined, return_tensors='pt', truncation=False)['input_ids']
            if tokens.shape[1] <= min(1024, pipeline.model_max_length):
                break
            re_chunks = chunker.chunk_text(combined)
            combined = " ".join(pipeline.summarize(rc) for rc in re_chunks)

        summaries.append(combined)

    # Clean up GPU memory
    del pipeline, model_config
    torch.cuda.empty_cache()
    gc.collect()

2025-05-04 20:35:30,064 - SummarizationPipeline - INFO - Initializing pipeline with model facebook/bart-large-cnn
2025-05-04 20:35:30,065 - SummarizationPipeline - INFO - Loading tokenizer for facebook/bart-large-cnn
2025-05-04 20:35:30,989 - SummarizationPipeline - INFO - Loading model for facebook/bart-large-cnn (8bit=False, device=cpu)
2025-05-04 20:35:31,747 - SummarizationPipeline - INFO - Model and tokenizer loaded successfully. Model max length: 1000000000000000019884624838656
2025-05-04 20:35:31,748 - SummarizationPipeline - INFO - Using prefix: summarize: 
2025-05-04 20:35:31,760 - SummarizationPipeline - WARNING - Model vocab size (50265) and tokenizer vocab size (50264) mismatch. Resizing model embeddings.
2025-05-04 20:35:33,049 - utils.text_chunker - INFO - Initialized TextChunker with chunk_size=1024, chunk_overlap=102, prefix=""


Summarizing with facebook/bart-large-cnn:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-04 20:35:33,111 - utils.text_chunker - INFO - Starting text chunking...
Chunking text: 100%|██████████| 13/13 [00:00<00:00, 70356.07it/s]
2025-05-04 20:35:33,133 - utils.text_chunker - INFO - Text successfully split into 13 chunks.
2025-05-04 20:35:33,134 - utils.text_chunker - INFO - Starting text chunking...


[4665, 4743, 4723, 4642, 4688, 4691, 4727, 4729, 4621, 4828, 4626, 4748, 3110]
chunk size: 1024


Chunking text: 100%|██████████| 76/76 [00:00<00:00, 283096.90it/s]
2025-05-04 20:35:33,149 - utils.text_chunker - INFO - Text successfully split into 76 chunks.
2025-05-04 20:35:33,151 - top2vec - INFO - Pre-processing documents for training
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-05-04 20:35:33,226 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


...

Chunking text: 100%|██████████| 152/152 [00:00<00:00, 529074.03it/s]
2025-05-04 21:06:30,588 - utils.text_chunker - INFO - Text successfully split into 152 chunks.
2025-05-04 21:06:30,590 - top2vec - INFO - Pre-processing documents for training
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-05-04 21:06:30,653 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


[611, 641, 526, 602, 592, 603, 589, 540, 560, 570, 553, 589, 612, 592, 586, 656, 592, 604, 616, 615, 587, 564, 580, 551, 557, 626, 579, 522, 538, 563, 529, 581, 662, 647, 620, 609, 582, 568, 594, 598, 602, 567, 570, 569, 550, 590, 613, 589, 568, 596, 608, 566, 555, 543, 540, 588, 610, 627, 578, 614, 646, 609, 590, 592, 569, 540, 553, 583, 613, 603, 574, 550, 565, 644, 658, 529, 613, 628, 539, 611, 582, 608, 587, 548, 554, 556, 548, 580, 609, 589, 588, 643, 579, 591, 614, 628, 592, 579, 577, 551, 543, 631, 595, 525, 537, 562, 556, 580, 669, 650, 639, 621, 617, 590, 605, 610, 584, 565, 540, 565, 526, 582, 606, 598, 590, 601, 600, 602, 546, 592, 565, 557, 602, 621, 611, 610, 637, 641, 587, 589, 581, 547, 567, 574, 584, 602, 600, 550, 567, 594, 677, 468]


2025-05-04 21:06:31,868 - top2vec - INFO - Creating joint document/word embedding
2025-05-04 21:06:33,366 - top2vec - INFO - Creating lower dimension embedding of documents
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-05-04 21:06:33,925 - top2vec - INFO - Finding dense areas of documents
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-05-04 21:06:33,933 - top2vec - INFO - Finding topics
Reducing topics: 100%|██████████| 1/1 [

Topic: market architecture growth software we year not so our great and just working really is do as work more now in what but at about very new are team see both terms re the that has this for to also ve you think it well with be right your of
found definition: pvgo: present value of growth opportunities.
found definition: present value of growth opportunities PVGO: Net present value of a firm™s future investments.
found definition: New growth theory: See GROWTH.


2025-05-04 21:07:03,391 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:07:31,860 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:08:05,652 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:08:36,254 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:09:05,138 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:09:38,462 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:10:09,311 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:10:39,339 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:11:11,660 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:11:45,666 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-05-04 21:12:14,861 - Summ

In [23]:
sting = ', '.join(summaries)

In [24]:
len(sting.split())

43

In [25]:
sting

'CEO Jensen Huang and CFO Colette Kress speak. Nvidia expects Hopper to ship in large volumes in April quarter Analysts: Is there a deal in the works with Mercedes-Benz Analysts: Can you give us a sense of where gaming is at right now'